In [1]:
import pandas as pd

# Besede in naglasi

Najprej preberemo seznam besed in naglašenih črk v besedah; te moramo nato pretvoriti v bolj oprijemljiv zapis.

Pri določanju zlogov poiščemo vse izolirane samoglasnike in jih proglasimo za svoj zlog. 
Samoglasniki, ki ne spadajo v ASCII, so poudarjeni (pri tem enako obravnamo tudi zlogotvorni _r_), ostali niso.

In [18]:
vse_besede = pd.read_csv("obdelani_podatki/besede.csv", names=["beseda", "poudarjena_beseda"])

In [19]:
vse_besede

,beseda,poudarjena_beseda
0,Pierre,Pierré
1,Pierra,Piérra
2,Pierrea,Pierréa
3,Pierru,Piérru
4,Pierreu,Pierréu
...,...,...
933101,zanj,zánj
933102,zanjo,zánjo
933103,zanju,zánju
933104,zanje,zánje


In [12]:
# Poiščimo vse črke v besedah, ki ne spadajo v ASCII
# Te potrebujemo, da lahko določimo, kaj je samoglasnik

def neznane_crke(beseda):
    return {x for x in beseda if ord(x) > 0x7f}

vse_neznanke = set()
for s in vse_besede['poudarjena_beseda'].apply(neznane_crke).values:
    vse_neznanke.update(s)

"".join(vse_neznanke)

'ŽđŔöžëĐµÁóÓŕÚšúćáŠčĆÖÍíČäüÉé'

In [21]:
def poudarjeni_zlogi(poudarjena_beseda):
    """Pretvori zapis s poudarjenimi znaki v zapis zlogov. Znak '-' pomeni poudarjen zlog, 'u' pa nepoudarjen.
        'zánju' -> '-u'
    """

    def je_samoglasnik(ch):
        return ch in 'aeiouAEIOUöëÁóÓŕÚŔúáÖÍíäüÉé'

    def je_poudarjen(samoglasnik):
        if len(samoglasnik) == 1:
            return samoglasnik not in "aeiou"
        else:
            return any(je_poudarjen(ch) for ch in samoglasnik)

    samoglasniki = []
    for i, ch in enumerate(poudarjena_beseda):
        if je_samoglasnik(ch):
            if i >= 1 and je_samoglasnik(poudarjena_beseda[i-1]):
                # samoglasnik ni izoliran
                samoglasniki[-1] += ch
            else:
                samoglasniki.append(ch)
    
    return "".join(map(lambda ch: '-' if je_poudarjen(ch) else 'u', samoglasniki))

vse_besede['zlogi'] = vse_besede.poudarjena_beseda.apply(poudarjeni_zlogi)
vse_besede

,beseda,poudarjena_beseda,zlogi
0,Pierre,Pierré,u-
1,Pierra,Piérra,-u
2,Pierrea,Pierréa,u-
3,Pierru,Piérru,-u
4,Pierreu,Pierréu,u-
...,...,...,...
933101,zanj,zánj,-
933102,zanjo,zánjo,-u
933103,zanju,zánju,-u
933104,zanje,zánje,-u


# Podatki

Sedaj preberemo podatke o virih, kategorijah in literarnih delih.

In [23]:
literarna_dela = pd.read_csv("obdelani_podatki/podatki.csv", names=["vir", "naslov", "avtor", "besedilo", "leto"])
literarna_dela

,vir,naslov,avtor,besedilo,leto
0,https://sl.wikisource.org/wiki/%22Grad%22_Turn...,"""Grad"" Turnska peč",Janko Orožen,"\nPod Tovstom je slikovita in ozka soteska, ki...",-1
1,https://sl.wikisource.org/wiki/%22Grad%22_na_D...,&quot;Grad&quot; na Dobrovlju,Janko Orožen,\nNa težko dostopnem vrhu Dobrovlja je stal ne...,-1
2,https://sl.wikisource.org/wiki/%22Grad%22_na_M...,&quot;Grad&quot; na Marnem,Janko Orožen,"\nOb cesti, ki vodi pod Marnim na Dol, je kape...",-1
3,https://sl.wikisource.org/wiki/%22Grad%22_na_V...,"""Grad"" na Vipoti",Janko Orožen,\nVipota! Kdo je ne pozna! S svojo piramidasto...,-1
4,https://sl.wikisource.org/wiki/%22Pisarska%22_...,"""Pisarska"" praviš",Stanko Vraz,"\n""Pisarska"", praviš, ""reč je ničemurna:\n\nde...",1952
...,...,...,...,...,...
14737,https://sl.wikisource.org/wiki/%E2%80%9EGrad%E...,„Grad“ v Skalah,Janko Orožen,\nNekoč se je širilo jezero od Gradišča do Ška...,-1
14738,https://sl.wikisource.org/wiki/%E2%80%9ESlov._...,"„Slov. plan. društvo"" z ozirom na naše narodno...",Ivan Vdovič,"Krasna je zemlja slovenska. Na mejah, ki dele ...",1895
14739,https://sl.wikisource.org/wiki/%E2%80%9EZur_Pf...,"""Zur pflege der slovensichen Schriftsprache an...",NaN,"Von Franz Žnideršič,\nk. k. Professor. Gorz. D...",1903
14740,https://sl.wikisource.org/wiki/%E2%80%A0Ivan_%...,†Ivan Škrjanec,NaN,"V petek, dne 17. julija je umrl v\nrodni hiši ...",1903


In [26]:
viri = pd.read_csv("obdelani_podatki/viri.csv", names=["povezava", "tip"])
viri

,povezava,tip
0,https://sl.wikisource.org/wiki/%22Grad%22_Turn...,knjiga
1,https://sl.wikisource.org/wiki/%22Grad%22_na_V...,knjiga
2,https://sl.wikisource.org/wiki/%22%C5%A0tok%22...,knjiga
3,https://sl.wikisource.org/wiki/A._Pretnarjevih...,poezija
4,https://sl.wikisource.org/wiki/Abecedni_seznam...,ljudsko slovstvo
...,...,...
625,https://sl.wikisource.org/wiki/%C5%BDolcpa%C5%...,zbrano delo
626,https://sl.wikisource.org/wiki/%C5%BDupanova_M...,antikvarno gradivo
627,https://sl.wikisource.org/wiki/%C5%BDivotopis_...,separat
628,https://sl.wikisource.org/wiki/%C5%BDolcpa%C5%...,zbrano delo


In [29]:
kategorije = pd.read_csv("obdelani_podatki/kategorije.csv", names=["povezava", "kategorija"])
kategorije

,povezava,kategorija
0,https://sl.wikisource.org/wiki/%22Grad%22_Turn...,Objavljeno s pisnim dovoljenjem
1,https://sl.wikisource.org/wiki/%22Grad%22_Turn...,Dela-G
2,https://sl.wikisource.org/wiki/%22Grad%22_na_D...,Objavljeno s pisnim dovoljenjem
3,https://sl.wikisource.org/wiki/%22Grad%22_na_D...,Dela-G
4,https://sl.wikisource.org/wiki/%22Grad%22_na_M...,Objavljeno s pisnim dovoljenjem
...,...,...
58791,https://sl.wikisource.org/wiki/%E2%80%A0Ivan_%...,Dela leta 1903
58792,https://sl.wikisource.org/wiki/%E2%80%A0P._Lev...,V javni lasti
58793,https://sl.wikisource.org/wiki/%E2%80%A0P._Lev...,Dela-P
58794,https://sl.wikisource.org/wiki/%E2%80%A0P._Lev...,Slovan (1902)
